<a href="https://colab.research.google.com/gist/codestar12/c99308b445e4c4cf6ae1b9608de32786/base_model_train_resnet_imagenette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_datasets

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import tensorflow as tf

from tensorflow.keras.applications import resnet
from tensorflow.keras.applications.resnet import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)


#used to fix bug in keras preprocessing scope
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
print("processed")

processed


In [3]:
IMAGE_SIZE = (224, 224)
TRAIN_SIZE = 9469
VALIDATION_SIZE = 3925
BATCH_SIZE = 64
NUM_CLASSES = 10

Dataset code

In [4]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented imageweights
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], 
                                         boxes=boxes, 
                                         box_indices=np.zeros(len(scales)), 
                                         crop_size=IMAGE_SIZE)
        # Return a random crop
        return crops[tf.random.uniform(shape=[], 
                                       minval=0, 
                                       maxval=len(scales), 
                                       dtype=tf.int32)]


    choice = tf.random.uniform(())

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

def normalize(input_image):
  return preprocess_input(input_image)

@tf.function
def load_image_train(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  # if tf.random.uniform(()) > 0.5:
  #   input_image = tf.image.flip_left_right(input_image)
  augmentations = [flip, color, zoom, rotate]
  for f in augmentations:
    input_image = tf.cond(tf.random.uniform(()) > 0.75, lambda: f(input_image), lambda: input_image)

  #input_image = preprocess_input(input_image)
  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

@tf.function
def load_image_test(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  #input_image = preprocess_input(input_image)

  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

In [5]:
tfds.disable_progress_bar()
dataset, info = tfds.load('imagenette', with_info=True)

In [6]:
for example in dataset['train'].take(1):
  print(example["label"])

tf.Tensor(2, shape=(), dtype=int64)


In [7]:
info

tfds.core.DatasetInfo(
    name='imagenette',
    version=0.1.0,
    description='Imagenette is a subset of 10 easily classified classes from the Imagenet
dataset. It was originally prepared by Jeremy Howard of FastAI. The objective
behind putting together a small version of the Imagenet dataset was mainly
because running new ideas/algorithms/experiments on the whole Imagenet take a
lot of time.

This version of the dataset allows researchers/practitioners to quickly try out
ideas and share with others. The dataset comes in three variants:

  * Full size
  * 320 px
  * 160 px
This dataset consists of the Imagenette dataset {size} variant.
',
    homepage='https://github.com/fastai/imagenette',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=13394,
    splits={
        'train': 12894,
        'validation': 500,
    },
    supervised_keys=('imag

make the upscaled cifar dataset

In [8]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train.shuffle(buffer_size=1000).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset['validation'].map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)









































































































































































































# New Section

In [9]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [10]:
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [11]:
#print('Compute dtype: %s' % policy.compute_dtype)
#print('Variable dtype: %s' % policy.variable_dtype)

In [12]:
model = resnet.ResNet101(include_top=True, weights=None, classes=10)

In [13]:
TRAIN_SIZE  // BATCH_SIZE

147

In [14]:
VALIDATION_SIZE // BATCH_SIZE

61

In [15]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=5, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(patience=15, min_delta=1e-2, verbose=1, restore_best_weights=True)

model.compile(tf.keras.optimizers.SGD(.1, momentum=0.9, nesterov=True), loss="categorical_crossentropy", metrics=['acc'])
model.fit(train_dataset,
          epochs=100, 
          steps_per_epoch=(TRAIN_SIZE//BATCH_SIZE), 
          validation_data=test_dataset, 
          validation_steps=(VALIDATION_SIZE//BATCH_SIZE),
          verbose=1,
          callbacks=[reduce_lr, early_stop]
)

Train for 147 steps, validate for 61 steps
Epoch 1/100
147/147 [==============================] - 80s 546ms/step - loss: 3.7055 - acc: 0.1021 - val_loss: 7157.9507 - val_acc: 0.0969
Epoch 2/100
147/147 [==============================] - 68s 464ms/step - loss: 2.2890 - acc: 0.1355 - val_loss: 28.3950 - val_acc: 0.0440
Epoch 3/100
147/147 [==============================] - 68s 460ms/step - loss: 2.1297 - acc: 0.2089 - val_loss: 2.0412 - val_acc: 0.2306
Epoch 4/100
147/147 [==============================] - 68s 461ms/step - loss: 2.0936 - acc: 0.2321 - val_loss: 2.6706 - val_acc: 0.1421
Epoch 5/100
147/147 [==============================] - 68s 461ms/step - loss: 2.0400 - acc: 0.2577 - val_loss: 1.9920 - val_acc: 0.2678
Epoch 6/100
147/147 [==============================] - 68s 461ms/step - loss: 2.0011 - acc: 0.2782 - val_loss: 1.8576 - val_acc: 0.3254
Epoch 7/100
147/147 [==============================] - 68s 461ms/step - loss: 1.9460 - acc: 0.3080 - val_loss: 1.7822 - val_acc: 0.3665
E

In [17]:
model.save(filepath='../imagenette.h5')

4/ugH8tvEReB5cvlCO8fqv4nxdyHVdwKynhQ0_RhOl1RIo2ZhGirH5mlM